In [70]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import date, timedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 999
pd.options.display.max_columns = 50

In [80]:
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

# Function that combines a date and time to produce a datetime.datetime
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])

# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)


def calculate_delayed(x):
    if x.days == 0:
        return x.seconds * -1
    if x.seconds > 50000:
        return 86400 - x.seconds
    else:
        return x.seconds * -1

In [3]:
df = pd.read_csv('flights_train.csv', parse_dates=False)

In [5]:
df2 = df.copy()

In [6]:
df2['DATE'] = pd.to_datetime(df2[['YEAR','MONTH', 'DAY']])
df2['SCHEDULED_DEPARTURE'] = create_flight_time(df2, 'SCHEDULED_DEPARTURE')
df2['DEPARTURE_TIME'] = df2['DEPARTURE_TIME'].apply(format_heure)
df2['SCHEDULED_ARRIVAL'] = df2['SCHEDULED_ARRIVAL'].apply(format_heure)
df2['SCHEDULED_DEPARTURE_TIME'] = df2['SCHEDULED_DEPARTURE'].dt.time
df2['t1'] = df2['SCHEDULED_DEPARTURE_TIME'].apply(lambda x: datetime.datetime.strptime(str(x),'%H:%M:%S'))
df2['t2'] = df2['DEPARTURE_TIME'].apply(lambda x: datetime.datetime.strptime(str(x),'%H:%M:%S'))
df2['DELAYED_DEPARTURE'] = df2['t1'] - df2['t2']
df2['DELAYED_DEPARTURE'] = df2['DELAYED_DEPARTURE'].apply(calculate_delayed)

In [7]:
df2.head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DATE,SCHEDULED_DEPARTURE_TIME
0,0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,21.0,15.0,205.0,1448,04:30:00,-22.0,2015-01-01,00:05:00
1,1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,00:02:00,12.0,14.0,280.0,2330,07:50:00,-9.0,2015-01-01,00:10:00
2,2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,00:18:00,16.0,34.0,286.0,2296,08:06:00,5.0,2015-01-01,00:20:00
3,3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,00:15:00,15.0,30.0,285.0,2342,08:05:00,-9.0,2015-01-01,00:20:00
4,4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,11.0,35.0,235.0,1448,03:20:00,-21.0,2015-01-01,00:25:00


In [ ]:
# sanity check

df2['DELAYED_DEPARTURE'][0] # -660
df2['DELAYED_DEPARTURE'][1] # -480
df2['DELAYED_DEPARTURE'][7] # 840
df2['DELAYED_DEPARTURE'][9] # 180
df2['DELAYED_DEPARTURE'][20] # 1500
df2['DELAYED_DEPARTURE'][36] # 180

In [87]:
columns_to_drop = ['YEAR', 'id', 'MONTH', 'DAY', 'FLIGHT_NUMBER', 'DESTINATION_AIRPORT', 'TAIL_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DATE', 'SCHEDULED_DEPARTURE_TIME', 't1', 't2']
df3 = df2.drop(columns_to_drop, axis = 1, inplace = False)

In [88]:
df3.head(5)

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DELAYED_DEPARTURE
0,4,AS,ANC,21.0,15.0,205.0,1448,04:30:00,-22.0,-660
1,4,AA,LAX,12.0,14.0,280.0,2330,07:50:00,-9.0,-480
2,4,US,SFO,16.0,34.0,286.0,2296,08:06:00,5.0,-120
3,4,AA,LAX,15.0,30.0,285.0,2342,08:05:00,-9.0,-300
4,4,AS,SEA,11.0,35.0,235.0,1448,03:20:00,-21.0,-60
